凯利公式（Kelly formula），也称凯利方程式，是一个用以使特定赌局中，拥有正期望值之重复行为长期增长率最大化的公式。这个公式由约翰·拉里·凯利于1956年在《贝尔系统技术期刊》中发表，可用以计算出每次游戏中应投注的资金比例。凯利公式是一种优化投资策略的公式，它可以帮助投资者在长期投资中最大化其收益。

##### 以投硬币为例，假设获胜200%收益，失败100%亏损，那应该下注多少呢？其期望值为2 * 0.5 +(-1) * 0.5 = 0.5 

In [36]:
import pandas as pd
## 1.1 每次100%投入多久会血本无归？ 
def cal_zero_cost_times(total_nums):
    count = 0
    win_times = 0
    ## 所有实验次数
    while count < total_nums:
        win_or_loss = np.random.uniform(0, 1)
        win_times = 0
        # 一直扔，直到赔光
        while win_or_loss >= 0.5:
            win_times = win_times + 1
            win_or_loss = np.random.uniform(0, 1)
        count = count + win_times
        # 返回平均计算，归0 的次数
    return count/total_nums
df = pd.DataFrame({})
test_numbsers = [1,10,1000,10000,100000,1000000]
df.insert(0, 'TestNumbers', test_numbsers)
df.insert(1, 'TestResults', test_numbsers) 
df['TestResults'] = df['TestResults'].apply(cal_zero_cost_times)
df

,TestNumbers,TestResults
0,1,2.00000
1,10,1.10000
2,1000,1.00300
3,10000,1.00050
4,100000,1.00002
5,1000000,1.00000


从超过1000次的平均来看，基本一笔就亏空了，这反向说明数据随机性。这种赌徒式押注，从长期看注定一无所有。

In [93]:
## 1.2 每次投入不同比例呢？ 
def cal_diff_percent_profit(test_percent, loop_times = 100,  test_cases = 10000,):
    count = 0
    ## 所有实验次数
    total_value = 0.0
    while count < test_cases:
        init_value = 1
        loop_number = 0 
        # test_number次测试
        while loop_number < loop_times:
            is_win = np.random.uniform(-1, 1)
            if is_win> 0:
                # 正收益
                init_value = init_value + init_value * test_percent
            else:
                # 亏了
                init_value = init_value - init_value *test_percent
            loop_number = loop_number + 1
            
        total_value = total_value + init_value
        count = count + 1
    return round(total_value/test_cases,2)
df = pd.DataFrame({})
df.insert(0, 'TestNumbers', np.arange(0.1,1,0.1))
df.insert(1, 'TestResults', np.arange(0.1,1,0.1)) 
df['TestResults'] = df['TestResults'].apply(cal_diff_percent_profit)
df

,TestNumbers,TestResults
0,0.1,1.00
1,0.2,1.05
2,0.3,1.00
3,0.4,0.64
4,0.5,0.04
5,0.6,0.03
6,0.7,0.00
7,0.8,0.00
8,0.9,0.00


从结果上看，这个盈亏比少下仓为妙。

In [94]:
## 1.3 凯利公式怎么说？ 
# 赔率是2，因为获胜将获得200%的收益，即赢得的金额是投注金额的两倍
def kelly_criterion(p_win, odds = 2.0):  
    """  
    使用凯利公式计算最佳投注比例。  
  
    参数:  
        p_win (float): 获胜的概率。  
        odds (float): 赔率，即赢得的金额与投注金额的比例。  
  
    返回:  
        float: 最佳投注比例。  
    """  
    # 失败的概率  
    p_lose = 1.0 - p_win  
  
    # 根据凯利公式计算最佳投注比例  
    return (p_win * odds - p_lose) / odds  
  
df = pd.DataFrame({})
df.insert(0, 'WinRate', np.arange(0.1,1,0.1))
df.insert(1, 'KellyBet', np.arange(0.1,1,0.1)) 
# 使用凯利公式计算最佳投注比例， 赔率从0.1 到0.9 都计算
df['KellyBet'] = df['KellyBet'].apply(kelly_criterion)
df

,WinRate,KellyBet
0,0.1,-0.35
1,0.2,-0.20
2,0.3,-0.05
3,0.4,0.10
4,0.5,0.25
5,0.6,0.40
6,0.7,0.55
7,0.8,0.70
8,0.9,0.85


In [81]:
# 只有胜率大于40的时候，凯利公式才给出了正值；在前面的例子中，胜率为0.5，赔率为2， 凯利公式推荐0.25，虽然
print("凯利公式计算结果收益：",cal_diff_percent_profit(0.25, 100,10000))

凯利公式计算结果收益： 1.05


##### 在股票中使用凯利公式的例子

In [141]:
# 2.1 假设有一个股票，估计它有60%的概率上涨5%，有40%的概率下跌2%。使用这些信息来计算凯利公式中的最优投资比例。
import math  
import akshare as ak

### 股票
##### 使用akshare 获取上证指数历史行情
df = ak.stock_zh_index_daily_em(symbol="sh000001", start_date="20200101", end_date="20210101")
#print(df)
df['Return'] = df['close'].pct_change()  
# 定义股票上涨和下跌的概率及收益率  

up_days = df[df['Return'] > 0]  
down_days = df[df['Return'] < 0]  
  
# 计算上涨和下跌的概率  
win_prob = len(up_days) / len(df)  
loss_prob = len(down_days) / len(df)  
# 计算收益
win_return = up_days['Return'].mean()  
loss_return = down_days['Return'].mean()  

# 盈亏比
reward_to_risk = abs(win_return / loss_return)  
# 计算最佳比率 
optimal_bet_fraction = kelly_criterion(win_prob, reward_to_risk)  
  
# 输出最优投资比例  
print(f"最优投资比例为：{optimal_bet_fraction:.2%}")  
  
# 假设你有10000元的资本，计算实际应该投入的金额  
capital = 100  
actual_bet_amount = capital * optimal_bet_fraction  
  
print(f"如果你有{capital}元的资本，你应该投入{actual_bet_amount:.2f}元进行这次交易。")


最优投资比例为：6.06%
如果你有100元的资本，你应该投入6.06元进行这次交易。


In [ ]:
# 2.2 